In [1]:
#data prefix for output data
yearmonth = "202405"

In [2]:
#import necessary packages
import platform
import os
import sys
import time
#for managing the data
import pandas as pd
from io import StringIO

#install !pip install webdriver-manager
#webdrivers
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By

from bs4 import BeautifulSoup

import pathlib

import os

In [4]:
#get current path
curr_path = os.getcwd()
project_tag = curr_path.split("/")[-1]
grouping = curr_path.split("/")[-2]
output_path = "../../../data/{}/{}/{}/".format(grouping,project_tag,yearmonth )

#create data output path if it doesn't exist
pathlib.Path(output_path).mkdir(parents=True, exist_ok=True) 
print("data output path created,", output_path)

data output path created, ../../../data/regional/brent_county_health_unit/202405/


In [38]:
#open web driver
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))

In [39]:
#url
base_url_list = \
[
    ['food',"https://inspectionreports.bchu.org/Disclosure/#/42635db8-42d9-47bb-b797-0bed66de3b46/disclosure", \
     ['Facility Name','Site Address','Facility Type','Last Inspection Date'] ],
    ['recreational_water',"https://inspectionreports.bchu.org/Disclosure/#/58e76e6b-2ba5-4772-bc1a-879c912952c3/disclosure", \
     ['Facility Name','Site Address','Facility Type','Last Inspection Date']],
    ['daycares',"https://inspectionreports.bchu.org/Disclosure/#/2ddb5467-6eb1-406f-8161-ad58c233f528/disclosure",\
     ['Facility Name','Site Address','Last Inspection Date']],
    ['temporarily_closed',"http://disclosure.algomapublichealth.com/Portal/Closure",\
     ['Facility Name','Site Address','Community','Program Area','Closed Date','Reason','Reopen Date']],
    ['tickets_orders',"http://disclosure.algomapublichealth.com/Portal/Enforcements",\
     ['Facility Name','Site Address','Type','Offense','Outcome','Offense Date','Resolved Date']],
]

In [40]:
#place holder for final dataframe list
df_list = []

for u in base_url_list:
    category = u[0]
    base_url = u[1]
    columns  = u[2]

    #load main landing page
    driver.get(base_url)
    
    try:
        #find disclaimer button
        disclamer_btn = driver.find_element(by='name', value="AcceptDisclaimerButton")

        #click the disclaimer button to move to the data page
        disclamer_btn.click()
    except:
        pass
    
    # We will loop through until there's no next page button on the webpage
    #find next page button
    last_page = False

    while last_page==False:
        try:
            next_btn = driver.find_element(By.CLASS_NAME, value="next-page-link")
        except:
            last_page = True


        #place holder for extracted data
        data_url_list = []

        #locate each table rows
        data_rows = driver.find_elements(By.CLASS_NAME, value="hovereffect")

        for row in data_rows:
            data_url_list.append( row.get_attribute("onclick") )

        #convert html table into dataframe
        df = pd.read_html(StringIO((driver.page_source)))
        df = df[0]
        

        #rename columns for clean up
        df.columns = columns
        
        #append url column and category column
        df['url_entity_partial'] = data_url_list
        df['category'] = category

        #put aside resulting table
        df_list.append(df)

        if last_page==True:
            break
        #move to the next page
        next_btn.click()

In [41]:
final_df = pd.concat(df_list)
print(final_df.shape)

(1387, 16)

In [46]:
#create full url link
url_prefix = "http://disclosure.algomapublichealth.com"

url_list=[]
for i,r in final_df.iterrows():
    temp_url_full = url_prefix + r['url_entity_partial'].split("'")[1]
    url_list.append( temp_url_full )

final_df['url_entity_full'] = url_list

In [47]:
final_df.to_csv( output_path + "1_level_table.csv", index=False)
print("file outputted to ",output_path + "1_level_table.csv")

In [48]:
#close webdriver
try:
    driver.close()
except:
    pass